In [1]:
ModelsDir <- '/home/kate/Research/Property/Models/'
DataDir <- '/home/kate/Research/Property/Data/'
ModelName <- "wc_nbglm_ofst_ecy"
UseSavedIfExists <- TRUE

In [2]:
library(MASS)

In [3]:
source('/home/kate/code/Utils/MyFunctions.R')

In [4]:
training_dataset <- read.csv(paste(DataDir,"property_wcf_training.csv", sep = ""), header=TRUE)
testing_dataset <- read.csv(paste(DataDir,"property_wcf_testing.csv", sep = ""), header=TRUE)
prediction_dataset <- read.csv(paste(DataDir,"property_water_claims_non_cat_fs.csv", sep = ""), header=TRUE)

In [5]:
formula <- cova_ic_nc_water ~ landlordind + log_sqft +  log_yearbuilt + roofcd + usagetype + cova_deductible + log_water_risk_3_blk

In [6]:
kfold <- 5

In [7]:
models_attr <- data.frame()
models_coef <- data.frame()
training_dataset$nbglm_ofst_ecy_cv <- 0
testing_dataset$nbglm_ofst_ecy_cv <- 0
prediction_dataset$nbglm_ofst_ecy_cv <- 0
for (fold in 1:kfold-1){
  training_dataset[paste("nbglm_ofst_ecy_",fold,sep="")] <- 0 
  testing_dataset[paste("nbglm_ofst_ecy_",fold,sep="")] <- 0  
  prediction_dataset[paste("nbglm_ofst_ecy_",fold,sep="")] <- 0    
    }
# 
#CV-Folds
AIC_l <- list();
BIC_l <- list();
logLik_l <- list();
coef_l <- list();
folds_l <- list() 
i <- 1
for (fold in 1:kfold-1){
    print(paste("Fold ",fold," of ",kfold," currently processing...",sep=""))
    training_dataset_fold <- training_dataset[training_dataset[paste("fold",fold,sep="_")] > 0,]
    validation_dataset <- training_dataset[training_dataset[paste("fold",fold,sep="_")] == 0,]
    #Modeling
    ModelFile <- paste(ModelsDir,ModelName,"_",fold,".rds",sep="")
    if(file.exists(ModelFile) && UseSavedIfExists){
        nbglm <- readRDS(ModelFile)
    } else {
        nbglm <- glm.nb(formula,  data = training_dataset_fold,  offset=log_ecy,control = glm.control(maxit=10000))
        saveRDS(nbglm, ModelFile)             
    }#if modeling 
    #Training Prediction
    pred_train <- predict(nbglm,training_dataset, type="response")
    training_dataset[paste("nbglm_ofst_ecy_",fold,sep="")] <- pred_train
    training_dataset$nbglm_ofst_ecy_cv <- training_dataset$nbglm_ofst_ecy_cv + pred_train/kfold
    #Testing Prediction
    pred_test <- predict(nbglm,testing_dataset, type="response")
    testing_dataset$nbglm_ofst_ecy_cv <- testing_dataset$nbglm_ofst_ecy_cv + pred_test/kfold 
    testing_dataset[paste("nbglm_ofst_ecy_",fold,sep="")] <- pred_test
    #Prediction
    pred <- predict(nbglm,prediction_dataset, type="response")
    prediction_dataset$nbglm_ofst_ecy_cv <- prediction_dataset$nbglm_ofst_ecy_cv + pred/kfold 
    prediction_dataset[paste("nbglm_ofst_ecy_",fold,sep="")] <- pred
    #Model attributes
    folds_l[[i]] <- i
    AIC_l[[i]] <- AIC(nbglm)
    BIC_l[[i]] <- BIC(nbglm)  
    logLik_l[[i]] <- logLik(nbglm) 
    #Model coeficients
    model_coef_fold <- data.frame(
        fold = fold+1,
        pred_name = summary(nbglm)$coefficients[,0],
        estimate =  summary(nbglm)$coefficients[,1],
        StdError =  summary(nbglm)$coefficients[,2],  
        zValue =   summary(nbglm)$coefficients[,3],     
        valuePr =   summary(nbglm)$coefficients[,4]    
    )  
    #pred_name is an index, row name, convert it into column
    model_coef_fold <- cbind(name = rownames(model_coef_fold), model_coef_fold)
    rownames(model_coef_fold) <- 1:nrow(model_coef_fold) 
    #main table
    models_coef <- rbind(models_coef,model_coef_fold)         
    i <- i + 1
    }#folds loop   
    #Model attributes data frame
    models_fold_attr <- data.frame( 
        fold = unlist(folds_l),
        AIC = unlist(AIC_l),
        BIC = unlist(BIC_l),   
        logLik = unlist(logLik_l)
    )
    models_attr <- rbind(models_attr,models_fold_attr)                 

[1] "Fold 0 of 5 currently processing..."
[1] "Fold 1 of 5 currently processing..."
[1] "Fold 2 of 5 currently processing..."
[1] "Fold 3 of 5 currently processing..."
[1] "Fold 4 of 5 currently processing..."


In [8]:
head(models_attr)

fold,AIC,BIC,logLik
1,91123.50,91351.67,-45542.75
2,91017.96,91246.14,-45489.98
3,90989.26,91217.43,-45475.63
4,90911.30,91139.48,-45436.65
5,90977.03,91205.21,-45469.52


In [9]:
write.table(models_attr,paste(ModelsDir,ModelName,"_attr.csv", sep = ""), sep=",",  col.names=TRUE, row.names = FALSE)

In [10]:
head(models_coef)

name,fold,estimate,StdError,zValue,valuePr
(Intercept),1,-120.71375039,9.67229248,-12.4803660,9.554642e-36
landlordind,1,-0.24210247,0.07387382,-3.2772433,1.048260e-03
log_sqft,1,0.57981663,0.03314515,17.4932595,1.612643e-68
log_yearbuilt,1,14.12805429,1.27845780,11.0508570,2.171322e-28
roofcdMEMBRANE,1,0.04831151,0.10437744,0.4628540,6.434690e-01
roofcdMETAL,1,0.11819722,0.22327666,0.5293756,5.965449e-01


In [11]:
write.table(models_coef,paste(ModelsDir,ModelName,"_coef.csv", sep = ""), sep=",",  col.names=TRUE, row.names = FALSE)

Normalized Weighted gini Training

In [12]:
NormalizedWeightedGini(training_dataset$cova_ic_nc_water,training_dataset$nbglm_ofst_ecy_cv,training_dataset$ecy)

[1] 0.38019

Normalized Weighted gini Testing

In [13]:
NormalizedWeightedGini(testing_dataset$cova_ic_nc_water,testing_dataset$nbglm_ofst_ecy_cv,testing_dataset$ecy)

[1] 0.403627

In [14]:
write.table(training_dataset,paste(DataDir,"property_wcf_training.csv", sep = ""), sep=",",  col.names=TRUE, row.names = FALSE)
write.table(testing_dataset,paste(DataDir,"property_wcf_testing.csv", sep = ""), sep=",",  col.names=TRUE, row.names = FALSE)
write.table(prediction_dataset,paste(DataDir,"property_water_claims_non_cat_fs.csv", sep = ""), sep=",",  col.names=TRUE, row.names = FALSE)